<a href="https://colab.research.google.com/github/pratyushharsh/deepLearning/blob/master/Digit_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf


In [ ]:
# Loading the MNSIT dataset using tensorflow dataset